# Run configuration

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# To be able to import .py scripts stored under src/
import sys
sys.path.append('/content/drive/MyDrive/Colab Notebooks/cmpe593/term-project/src')
import os
import torch
from datetime import datetime


# Run parameters
model_type = "custom_FADE" # TODO: custom_FADE or "baseline"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

##############################################################################
# PATHS
##############################################################################
base_path = "/content/drive/MyDrive"
project_dir = os.path.join(base_path, "Colab Notebooks", "cmpe593", "term-project")

## Dataset paths
dataset_split = "val2017"
coco_base_dir = f'{base_path}/coco'
images_dir = os.path.join(coco_base_dir, 'images', dataset_split)
annotations_path = os.path.join(coco_base_dir,
                                'annotations',
                                f'instances_{dataset_split}.json')

## Artifact paths (checkpoints, plots, metrics)
timestamp = "20250105_164623" # TODO: Enter the timestamp of the model checkpoint you want to import
results_dir = os.path.join(project_dir, 'results', timestamp)
checkpoint_dir = os.path.join(results_dir, 'checkpoints')
plot_dir = os.path.join(results_dir, 'plots')
metrics_dir = os.path.join(results_dir, 'metrics')


# Ensure artifact paths exist
for dir in [results_dir, checkpoint_dir, plot_dir, metrics_dir]:
  if not os.path.exists(dir):
    os.makedirs(dir)
    print(f'Directory created at {dir}')
  else:
    print(f'Directory {dir} already exists.')
##############################################################################

model_full_path = os.path.join(checkpoint_dir, "model_final.pth")
eval_results_path = os.path.join(metrics_dir, "evaluation_metrics.json")

Directory /content/drive/MyDrive/Colab Notebooks/cmpe593/term-project/results/20250105_164623 already exists.
Directory /content/drive/MyDrive/Colab Notebooks/cmpe593/term-project/results/20250105_164623/checkpoints already exists.
Directory /content/drive/MyDrive/Colab Notebooks/cmpe593/term-project/results/20250105_164623/plots already exists.
Directory /content/drive/MyDrive/Colab Notebooks/cmpe593/term-project/results/20250105_164623/metrics already exists.


# Main loop

In [ ]:
from modelops.model_loader import load_model
from dataops.data_loader import get_data_loader, get_ground_truth

# Load the model
model = load_model(model_type, model_full_path, device, pretrained=False)

# Get data loader
dataloader = get_data_loader(images_dir, annotations_path, train=False)

Creating custom FADE model...


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 190MB/s]


Loading custom FADE model weights from /content/drive/MyDrive/Colab Notebooks/cmpe593/term-project/results/20250105_164623/checkpoints/model_final.pth


/content/drive/MyDrive/Colab Notebooks/cmpe593/term-project/src/modelops/model_loader.py:93: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(model_path

Custom FADE model loaded.
Loaded custom_FADE model for evaluation on cuda.
Drive already mounted at /content/drive.

Image file directory: /content/drive/MyDrive/coco/images/val2017
Does image directory exist? True

Annotation file path: /content/drive/MyDrive/coco/annotations/instances_val2017.json
Does annotation file exist? True
Drive already mounted at /content/drive.

Generating image IDs...


Processing images: 100%|██████████| 5000/5000 [00:00<00:00, 1080393.59file/s]


Total of 5000 image IDs were generated.
loading annotations into memory...
Done (t=1.25s)
creating index...
index created!

Dataset size is 5000
Creating dataloader object...


In [ ]:
from modelops.inference import predict

# Get predictions
results = predict(model, dataloaders, device)

# Load predictions into COCO format
# coco _dt: detections, _gt: ground truth
coco_gt = get_ground_truth(annotations_path)
coco_dt = coco_gt.loadRes(results)

loading annotations into memory...
Done (t=0.88s)
creating index...
index created!


100%|██████████| 5000/5000 [11:16<00:00,  7.39it/s]


In [ ]:
# Create COCOeval object
from pycocotools.cocoeval import COCOeval
coco_evaluator = COCOeval(coco_gt, coco_dt, iouType='bbox')

# Run evaluation
coco_evaluator.evaluate()    # Run per-image evaluation
coco_evaluator.accumulate()  # Aggregate the results
coco_evaluator.summarize()   # Print the summary metrics

Loading and preparing results...
DONE (t=0.66s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=24.39s).
Accumulating evaluation results...
DONE (t=6.54s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.010
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.024
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.006
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.006
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.011
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.014
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.024
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.041
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.043
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=1

In [ ]:
import json
# Extract key metrics from COCOeval
coco_metrics = {
    "Average Precision (AP) @[IoU=0.50:0.95 | area=all | maxDets=100]": coco_evaluator.stats[0],
    "Average Precision (AP) @[IoU=0.50      | area=all | maxDets=100]": coco_evaluator.stats[1],
    "Average Precision (AP) @[IoU=0.75      | area=all | maxDets=100]": coco_evaluator.stats[2],
    "Average Precision (AP) @[IoU=0.50:0.95 | area=small | maxDets=100]": coco_evaluator.stats[3],
    "Average Precision (AP) @[IoU=0.50:0.95 | area=medium | maxDets=100]": coco_evaluator.stats[4],
    "Average Precision (AP) @[IoU=0.50:0.95 | area=large | maxDets=100]": coco_evaluator.stats[5],
    "Average Recall (AR) @[IoU=0.50:0.95 | area=all | maxDets=1]": coco_evaluator.stats[6],
    "Average Recall (AR) @[IoU=0.50:0.95 | area=all | maxDets=10]": coco_evaluator.stats[7],
    "Average Recall (AR) @[IoU=0.50:0.95 | area=all | maxDets=100]": coco_evaluator.stats[8],
    "Average Recall (AR) @[IoU=0.50:0.95 | area=small | maxDets=100]": coco_evaluator.stats[9],
    "Average Recall (AR) @[IoU=0.50:0.95 | area=medium | maxDets=100]": coco_evaluator.stats[10],
    "Average Recall (AR) @[IoU=0.50:0.95 | area=large | maxDets=100]": coco_evaluator.stats[11],
}

# Save to JSON file
with open(eval_results_path, "w") as f:
    json.dump(coco_metrics, f, indent=4)

print(f"Evaluation metrics saved to {eval_results_path}")

Evaluation metrics saved to /content/drive/MyDrive/Colab Notebooks/cmpe593/term-project/results/20250105_164623/metrics/evaluation_metrics.json


In [ ]:
print(results[:5])

[{'image_id': 139, 'category_id': 1, 'bbox': [335.3409118652344, 184.2754364013672, 127.52828979492188, 138.2395477294922], 'score': 0.1523757427930832, 'segmentation': [[335.3409118652344, 184.2754364013672, 335.3409118652344, 322.5149841308594, 462.86920166015625, 322.5149841308594, 462.86920166015625, 184.2754364013672]], 'area': 17629.453103965614, 'id': 1, 'iscrowd': 0}, {'image_id': 139, 'category_id': 1, 'bbox': [364.6128845214844, 223.4462432861328, 115.8565673828125, 142.7678985595703], 'score': 0.14644700288772583, 'segmentation': [[364.6128845214844, 223.4462432861328, 364.6128845214844, 366.2141418457031, 480.4694519042969, 366.2141418457031, 480.4694519042969, 223.4462432861328]], 'area': 16540.598659569398, 'id': 2, 'iscrowd': 0}, {'image_id': 139, 'category_id': 1, 'bbox': [276.9834289550781, 209.77529907226562, 138.9505615234375, 151.67990112304688], 'score': 0.14604786038398743, 'segmentation': [[276.9834289550781, 209.77529907226562, 276.9834289550781, 361.45520019531

In [ ]:
cleaned_results = [
    {
        "image_id": r["image_id"],
        "category_id": r["category_id"],
        "bbox": r["bbox"],
        "score": r["score"]
    }
    for r in results
]

print("Cleaned Results:", cleaned_results[:5])

Cleaned Results: [{'image_id': 139, 'category_id': 1, 'bbox': [335.3409118652344, 184.2754364013672, 127.52828979492188, 138.2395477294922], 'score': 0.1523757427930832}, {'image_id': 139, 'category_id': 1, 'bbox': [364.6128845214844, 223.4462432861328, 115.8565673828125, 142.7678985595703], 'score': 0.14644700288772583}, {'image_id': 139, 'category_id': 1, 'bbox': [276.9834289550781, 209.77529907226562, 138.9505615234375, 151.67990112304688], 'score': 0.14604786038398743}, {'image_id': 139, 'category_id': 1, 'bbox': [260.7035217285156, 187.41854858398438, 118.7919921875, 126.1197509765625], 'score': 0.1425986886024475}, {'image_id': 139, 'category_id': 1, 'bbox': [334.6036376953125, 122.29469299316406, 210.81414794921875, 278.064453125], 'score': 0.13756239414215088}]


In [ ]:
coco_dt = coco_gt.loadRes(cleaned_results)
coco_evaluator = COCOeval(coco_gt, coco_dt, iouType='bbox')
coco_evaluator.evaluate()
coco_evaluator.accumulate()
coco_evaluator.summarize()

Loading and preparing results...
DONE (t=1.02s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=22.29s).
Accumulating evaluation results...
DONE (t=3.68s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.000
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.001
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.001
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.001
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.001
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=1

In [ ]:
confidence_threshold = 0.5
filtered_results = [
    r for r in cleaned_results if r['score'] > confidence_threshold
]

print("Filtered Results Count:", len(filtered_results))

Filtered Results Count: 0
